Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In this colab, we demonstrate how [secret sharer](https://arxiv.org/abs/1802.08232) works on a language model. We will train a model with "secrets", i.e. random sequences, inserted in the training data, and then evaluate if the model has "memorized" those secrets.

(The code is largely based on a Tensorflow tutorial for training a character-level LSTM on Shakespeare dataset.)

#Setup

You may set the runtime to use a GPU by Runtime > Change runtime type > Hardware accelerator.

In [ ]:
# @title Install dependencies
# You may need to restart the runtime to use tensorflow-privacy.
from IPython.display import clear_output

!pip install tf-models-official
!pip install -e git+https://github.com/tensorflow/privacy.git
clear_output()

In [ ]:
import functools
import os
import numpy as np
import tensorflow as tf
from official.utils.misc import keras_utils

from tensorflow_privacy.privacy.privacy_tests.secret_sharer.generate_secrets import SecretConfig, generate_secrets_and_references, construct_secret
from tensorflow_privacy.privacy.privacy_tests.secret_sharer.exposures import compute_exposure_interpolation, compute_exposure_extrapolation

# Define functions

In [ ]:
#@title Define some parameters for the model.
EMBEDDING_DIM = 256
RNN_UNITS = 1024
SEQ_LENGTH = 100

BATCHES_PER_EPOCH = 11043
TRAIN_EPOCHS = 50
BATCH_SIZE = 64
LOG_STEPS = 100

ENABLE_EAGER = True

# We will download shakespeare.txt later
TRAINING_DATA = 'shakespeare.txt'
# We will try multiple sets of secrets, so we set a placeholder for the secret's name in the directory
MODEL_DIR = 'secret{}'

In [ ]:
#@title Function for getting dataset combined with secrets.
def get_dataset(path_to_file, secrets=None,
                batch_size=None, seq_length=SEQ_LENGTH):
  """Creates a dataset from a given text file.

  Args:
    path_to_file: The path to the training data.
    secrets: Secrets to be inserted.
    batch_size: Batch size to use.
    seq_length: The length of the LSTM sequence.

  Returns:
    A tuple, consisting of the Dataset and the class to character mapping
    and character to class mapping.
  """
  with tf.io.gfile.GFile(path_to_file, 'rb') as train_data:
    text = train_data.read().decode(encoding='utf-8')

  # Create vocab
  vocab = sorted(set(text))
  char2idx = {u: i for i, u in enumerate(vocab)}
  idx2char = np.array(vocab)

  # Gather all secrets
  if not secrets:
    pass
  else:
    all_secrets = []
    for secret in secrets:
      secret.datasets = {}
      for r, seqs in secret.secrets.items():
        all_secrets += seqs * r
        indexed_seqs = np.array([[char2idx[c] for c in s] for s in seqs])
        secret.datasets[r] = {
            'data': tf.data.Dataset.from_tensor_slices(indexed_seqs).map(lambda chunk: chunk[:-1]).batch(1),
            'label': list(map(lambda chunk: chunk[1:], indexed_seqs))
        }
      indexed_seqs = np.array([[char2idx[c] for c in s] for s in secret.references])
      secret.datasets[0] = {
          'data': tf.data.Dataset.from_tensor_slices(indexed_seqs).map(lambda chunk: chunk[:-1]).batch(1),
          'label': list(map(lambda chunk: chunk[1:], indexed_seqs))
      }
    # Insert the secrets into the training data
    idx = np.random.choice(len(text) - 1, len(all_secrets) - 1, replace=False) + 1
    # split text into len(all_secrets) parts
    text = np.split(list(text), np.sort(idx))
    text = [''.join(t) for t in text]
    text = ''.join(np.vstack([text, all_secrets]).transpose().reshape(-1))

  # Split text into sequence length + 1 chucks to create examples
  def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, tf.one_hot(target_text, len(vocab))

  text_as_int = np.array([char2idx[c] for c in text])
  char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
  sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

  dataset = sequences.map(split_input_target)
  dataset = dataset.shuffle(10000).repeat()
  dataset = dataset.batch(batch_size, drop_remainder=True)

  return dataset, idx2char, char2idx

In [ ]:
#@title Functions for building, training model and making prediction.
def build_model(vocab_size,
                embedding_dim=EMBEDDING_DIM,
                rnn_units=RNN_UNITS,
                batch_size=None,
                stateful=False,
                use_cudnn=True):
  """Builds the Shakespeare model.

  Args:
    vocab_size: The number of character classes in the input.
    embedding_dim: The dimension of the embedding space for each class.
    rnn_units: The number of RNN units in the layer.
    batch_size: When predicting, the batch size of the predictions.
    stateful: If true, the LSTM is stateful.

  Returns:
    A Keras Model.
  """
  LSTM = functools.partial(tf.keras.layers.LSTM, implementation=2)

  # By indirecting the activation through a lambda layer, the logic to dispatch
  # to CuDNN in V2 doesn't trigger and we force the LSTM to run in non-CuDNN
  # mode.
  lstm_activation = ('tanh' if use_cudnn else
                     lambda x: tf.math.tanh(x))

  batch_shape = [batch_size if stateful else None, None]
  return tf.keras.Sequential([
      tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                batch_input_shape=batch_shape),
      LSTM(rnn_units,
           activation=lstm_activation,
           return_sequences=True,
           stateful=stateful,
           recurrent_initializer='glorot_uniform'),
      tf.keras.layers.Dense(vocab_size),
      tf.keras.layers.Softmax(dtype=tf.float32)])


def train_model(dataset, vocab_size, batch_size,
                checkpoint_dir=None,
                use_cudnn=True):
  """Trains a Shakespeare model.

  Args:
    dataset: the training data set.
    vocab_size: the number of unique character classes.
    batch_size: batch size of training.
    checkpoint_dir: if not None, the directory in which to make checkpoints.

  Returns:
    The training history and callbacks.
  """
  train_steps = BATCHES_PER_EPOCH // batch_size

  model = build_model(vocab_size=vocab_size, batch_size=batch_size,
                      use_cudnn=use_cudnn)

  # When keras_use_ctl is False, Model.fit() automatically applies
  # loss scaling so we don't need to create a LossScaleOptimizer.
  model.compile(
      optimizer=tf.keras.optimizers.Adam(),
      loss=tf.keras.losses.CategoricalCrossentropy(),
      metrics=[tf.keras.metrics.Recall(top_k=1, name='RecallAt1'),
                tf.keras.metrics.Recall(top_k=5, name='RecallAt5')],
      run_eagerly=ENABLE_EAGER)

  callbacks = []
  if checkpoint_dir:
    checkpoint_filepath = os.path.join(checkpoint_dir, 'ckpt_{epoch}')
    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_filepath,
        save_weights_only=True)
    callbacks.append(checkpoint_callback)
    checkpoint_latest = tf.train.latest_checkpoint(checkpoint_dir)
    if checkpoint_latest:
      model.load_weights(checkpoint_latest)
      initial_epoch = int(os.path.basename(checkpoint_latest).split('.')[0].split('_')[-1])
      print(f'Loaded from {checkpoint_latest}. Will train from epoch {initial_epoch}.')
    else:
      initial_epoch = 0

  time_callback = keras_utils.TimeHistory(batch_size, LOG_STEPS)
  callbacks.append(time_callback)
  history = model.fit(dataset,
                      epochs=TRAIN_EPOCHS,
                      initial_epoch=initial_epoch,
                      steps_per_epoch=train_steps,
                      callbacks=callbacks,
                      verbose=2)
  return history, callbacks

In [ ]:
#@title Function to compute perplexity.
def compute_perplexity_for_secret(secrets, checkpoint_dir, vocab_size):
  """Get perplexity for given examples.

  Args:
    secrets: a list, where each element is a secret dictionary
    checkpoint_dir: the directory from which to load checkpoints
    vocab_size: vocabulary size

  Returns:
    Perplexity for the input examples.
  """
  prediction_model = build_model(
      vocab_size=vocab_size, batch_size=1, stateful=True)
  prediction_model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
  prediction_model.compile(loss=tf.keras.losses.CategoricalCrossentropy())

  cce = tf.keras.losses.SparseCategoricalCrossentropy(reduction=tf.keras.losses.Reduction.NONE)
  ppls = [{} for _ in range(len(secrets))]
  ppls_reference = [None] * len(secrets)
  for i, secret in enumerate(secrets):
    for r in secret.datasets.keys():
      d = secret.datasets[r]
      loss = np.mean(cce(d['label'], prediction_model.predict(d['data'])),
                     axis=1)
      loss = list(loss)
      if r == 0:
        ppls_reference[i] = loss
      else:
        ppls[i][r] = loss
  return ppls, ppls_reference

# Run training and secret sharer on Shakespeare dataset.

In [ ]:
#@title Donwload the Shakespeare dataset
!wget --no-check-certificate --no-verbose https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt
!head -n 20 shakespeare.txt

In [ ]:
#@title Check the vocabulary of the dataset
# (to make sure the secrets' vocabulary is a subset)
with open('shakespeare.txt', 'rb') as f:
  text = f.read().decode(encoding='utf-8')
  print(sorted(set(text)))

In [ ]:
#@title Generate secrets
# Here we present just one example and you can try other combinations of patterns,
# vocabs and number of repetitions. For example, you may use other patterns and
# different vocabularies for them.
patterns = ['{}' * 5,
            'Info ' + '{}' * 5,
            '{}{}-{}{}-{}',
            '{}' * 10]
vocabs = [list('ABCDEFGHIJ')] * 4
num_repetitions = [1, 10, 100]
num_secrets_for_repetitions = [20] * len(num_repetitions)
num_references = 65536
secret_configs = [SecretConfig(vocab, pattern, num_repetitions,
                              num_secrets_for_repetitions, num_references)
                  for vocab, pattern in zip(vocabs, patterns)]
secrets = generate_secrets_and_references(secret_configs)

# Let's look at the variable "secrets"
print(f'"secrets" is a list and the length is {len(secrets)} because we have four sets of secrets.')
print(f'The type of the element in "secrets" is {type(secrets[0])}.')
print(f'The field "config" contains its configuration: {secrets[0].config}')
print(f'The filed "secrets" is a dictionary, with key being the number of repetitions and value being a list of secrets.',
      f'For example, the secrets that are going to be repeated for 100 times are {secrets[0].secrets[100]}.')
print(f'The field "references" is a list of reference sequences that are not going to be injected. The first 10 elements are {secrets[0].references[:10]}.')

In [ ]:
#@title Combine secrets with training data and train the models.
# We're going to train 4 models corresponding to the four sets of secrets,
# to examine the effect of different ways of generating secrets.
for i in range(len(secrets)):
  dataset, idx2char, char2idx = get_dataset(
      TRAINING_DATA,
      [secrets[i]],
      batch_size=BATCH_SIZE)

  print(f'Train model with secret set #{i}.')
  train_model(dataset, len(idx2char), BATCH_SIZE,
              checkpoint_dir=MODEL_DIR.format(i))

In [ ]:
#@title Secret sharer evaluation
ppls_all, ppls_reference_all = [], []
for i in range(len(secrets)):
  ppls, ppls_reference = compute_perplexity_for_secret(
      [secrets[i]], MODEL_DIR.format(i), len(idx2char))
  ppls_all += ppls
  ppls_reference_all += ppls_reference

for i, (vocab, pattern, p, p_r) in enumerate(zip(vocabs, patterns, ppls_all, ppls_reference_all)):
  exposure_interpolation = compute_exposure_interpolation(p, p_r)
  exposure_extrapolation = compute_exposure_extrapolation(p, p_r)

  # exposure_interpolation (or exposure_extrapolation) is a dictionary, where the key is the number of repetitions and
  # the value is a list of exposures (not necessarily in the same order as the secrets). We will now take a look at
  # the average exposure (with standard deviation).
  print(f'Secrets #{i+1}, vocab={vocab}, pattern={pattern}')
  print('  Interpolation:', '; '.join([f'repetition={r}, avg_exposure={np.mean(exposure_interpolation[r]):.2f}±{np.std(exposure_interpolation[r]):.2f}' for r in num_repetitions]))
  print('  Extrapolation:', '; '.join([f'repetition={r}, avg_exposure={np.mean(exposure_extrapolation[r]):.2f}±{np.std(exposure_extrapolation[r]):.2f}' for r in num_repetitions]))

Secrets #1, vocab=['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'], pattern={}{}{}{}{}
  Interpolation: repetition=1, avg_exposure=1.40±1.05; repetition=10, avg_exposure=3.47±1.70; repetition=100, avg_exposure=6.40±2.97
  Extrapolation: repetition=1, avg_exposure=1.41±1.08; repetition=10, avg_exposure=3.59±1.82; repetition=100, avg_exposure=6.56±2.91
Secrets #2, vocab=['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'], pattern=Info {}{}{}{}{}
  Interpolation: repetition=1, avg_exposure=2.82±1.75; repetition=10, avg_exposure=7.41±2.96; repetition=100, avg_exposure=8.49±3.27
  Extrapolation: repetition=1, avg_exposure=2.81±1.72; repetition=10, avg_exposure=7.07±2.54; repetition=100, avg_exposure=7.95±2.77
Secrets #3, vocab=['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'], pattern={}{}-{}{}-{}
  Interpolation: repetition=1, avg_exposure=2.47±1.53; repetition=10, avg_exposure=3.42±2.41; repetition=100, avg_exposure=5.23±3.55
  Extrapolation: repetition=1, avg_exposure=2.49±1.52; repeti

We can see the following:
1. Comparing #1 vs. #2, having the prefix (#2) for the random sequences might help the model memorize.
2. Comparing #1 vs. #3, having the pattern (#3) did not help the memorization significantly. This is different from the result in the [paper](https://arxiv.org/abs/1802.08232). The effect might be model / data specific.
3. Comparing #1 vs. #4, longer secrets (#4) show higher exposure.

You can also try some other secret configurations to see if there is any difference.